In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

# importing libraries

In [ ]:
import detectron2
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.modeling import GeneralizedRCNNWithTTA, GeneralizedRCNN
from detectron2.config import get_cfg
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fastcore.all import *

from detectron2.utils.visualizer import Visualizer
import albumentations as A
from scipy import stats

detectron2.__version__

# Functions

In [ ]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def get_masks(fn, predictor):
    im = cv2.imread(str(fn))
    pred = predictor(im)
    
    
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(im.shape[:2], dtype=int) 
    for mask in pred_masks:
        mask = mask * (1 - used)
        if mask.sum() >= MIN_PIXELS[pred_class]: # skip predictions with small area
            used += mask
            res.append(rle_encode(mask))
    return res

In [ ]:
Dir_testdata = Path('../input/sartorius-cell-instance-segmentation')
ids, masks = [], []
test_image_names = (Dir_testdata/'test').ls()

# Load Model

In [ ]:
cfg = get_cfg()
# config_name = "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
config_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(config_name))

cfg.MODEL.WEIGHTS = "../input/sartoriousresnet50/model_0008953-DE10k-Au9k-1fold-0.2738.pth"


cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.40
cfg.TEST.DETECTIONS_PER_IMAGE = 1000


predictor = DefaultPredictor(cfg)
THRESHOLDS = [.15, .35, .55]  #- score 0.301

MIN_PIXELS = [75, 150, 75]

# Predicting

In [ ]:
for fn in test_image_names:
    encoded_masks = get_masks(fn, predictor)
    for enc in encoded_masks:
        ids.append(fn.stem)
        masks.append(enc)

In [ ]:
print("id",",","predicted")
print(ids[0],",",masks[0])

# Visualize predictions

In [ ]:
encoded_masks = get_masks(test_image_names[0], predictor)
_, axs = plt.subplots(1,2, figsize=(40, 30))
axs[1].imshow(cv2.imread(str(test_image_names[0])))
axs[1].axis("off")
mask = np.zeros((520, 704, 1))
for enc in encoded_masks:
    mask += rle_decode(enc, shape=(520, 704, 1))
    
mask = mask.clip(0, 1)
axs[0].imshow(mask)
axs[0].axis("off")

# Submission

In [ ]:
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()